In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.diagnostic import het_breuschpagan
from linearmodels.panel import PooledOLS, RandomEffects
from linearmodels.panel import compare
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv("D:/PENS22/SEMESTER 5/Ekonometrika/Data Ekonometrika/Ekonomi Sumut 2017-2022.csv")
df

,Kabupaten/Kota,Tahun,Indeks Pembangunan Manusia,Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010,Pengeluaran per Kapita Disesuaikan (Ribu Rupiah),Gini Ratio,Rata-Rata Lama Sekolah,TPT,Persentase Penduduk Miskin
0,Asahan,2017,69.10,4.60,10477,"0,2668",8.46,"5,95",11.67
1,Asahan,2018,69.49,4.78,10735,"0,2912",8.47,"5,26",10.25
2,Asahan,2019,69.92,4.53,10983,"0,2793",8.49,"6,86",9.68
3,Asahan,2020,72.35,-4.42,10890,"0,2605",8.79,"7,24",9.04
4,Asahan,2021,72.55,2.12,11030,"0,2441",8.80,"6,39",9.35
...,...,...,...,...,...,...,...,...,...
193,Toba Samosir,2018,74.48,4.45,12095,"0,3279",10.34,"2,15",8.67
194,Toba Samosir,2019,74.92,4.34,12375,"0,2753",10.36,"1,26",8.60
195,Toba Samosir,2020,76.81,-10.85,12154,"0,2903",10.52,"2,5",8.71
196,Toba Samosir,2021,77.03,1.59,12224,"0,2719",10.57,"0,83",8.99


In [3]:
print(df.dtypes)

Kabupaten/Kota                                                                               object
Tahun                                                                                         int64
Indeks Pembangunan Manusia                                                                  float64
Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010    float64
Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)                                              int64
Gini Ratio                                                                                   object
Rata-Rata Lama Sekolah                                                                      float64
TPT                                                                                          object
Persentase Penduduk Miskin                                                                  float64
dtype: object


In [4]:
df['Gini Ratio'] = df['Gini Ratio'].str.replace(',', '.').astype(float)
df['TPT'] = df['TPT'].str.replace(',', '.').astype(float)

print(df.dtypes)


Kabupaten/Kota                                                                               object
Tahun                                                                                         int64
Indeks Pembangunan Manusia                                                                  float64
Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010    float64
Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)                                              int64
Gini Ratio                                                                                  float64
Rata-Rata Lama Sekolah                                                                      float64
TPT                                                                                         float64
Persentase Penduduk Miskin                                                                  float64
dtype: object


# Uji Asumsi Klasik

In [5]:
y = df['Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)']  # Variabel dependen
X = df[[
        'Persentase Penduduk Miskin', 
        'Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010', 
        'Gini Ratio', 
        'Rata-Rata Lama Sekolah', 
        'TPT']]

X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
residuals = model.resid

print(model.summary())

                                           OLS Regression Results                                           
Dep. Variable:     Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)   R-squared:                       0.737
Model:                                                          OLS   Adj. R-squared:                  0.730
Method:                                               Least Squares   F-statistic:                     107.4
Date:                                              Fri, 15 Nov 2024   Prob (F-statistic):           1.12e-53
Time:                                                      17:17:59   Log-Likelihood:                -1659.2
No. Observations:                                               198   AIC:                             3330.
Df Residuals:                                                   192   BIC:                             3350.
Df Model:                                                         5                                         
Covariance Type:   

# Uji Heterokedastisitas

In [25]:
# Uji heteroskedastisitas menggunakan Breusch-Pagan
_, pval_bp, _, _ = het_breuschpagan(residuals, model.model.exog)

print('Breusch-Pagan Test Statistics:')
print(pval_bp)

# Interpretasi hasil uji heteroskedastisitas
if pval_bp < 0.05:
    print("Terdapat bukti untuk menolak hipotesis nol, menunjukkan adanya heteroskedastisitas.")
else:
    print("Tidak ada cukup bukti untuk menolak hipotesis nol, menunjukkan tidak ada heteroskedastisitas.")

Breusch-Pagan Test Statistics:
0.23071261767736764
Tidak ada cukup bukti untuk menolak hipotesis nol, menunjukkan tidak ada heteroskedastisitas.


# Chow Test

In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import f

# Filter data berdasarkan tahun 2018-2019 dan 2020-2022
df1 = df[df['Tahun'].isin([2017,2018, 2019])]  # Subset data untuk tahun 2018 dan 2019
df2 = df[df['Tahun'].isin([2020, 2021, 2022])]  # Subset data untuk tahun 2020-2022

# Definisikan variabel dependen dan independen untuk setiap subset
y1 = df1['Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)']
X1 = df1[[
        'Persentase Penduduk Miskin', 
        'Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010', 
        'Gini Ratio', 
        'Rata-Rata Lama Sekolah', 
        'TPT']]
X1 = sm.add_constant(X1)  # Menambahkan konstanta (intercept)

y2 = df2['Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)']
X2 = df2[[
        'Persentase Penduduk Miskin', 
        'Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010', 
        'Gini Ratio', 
        'Rata-Rata Lama Sekolah', 
        'TPT']]
X2 = sm.add_constant(X2)  # Menambahkan konstanta (intercept)

# Regresi OLS untuk subset pertama (2018-2019)
model1 = sm.OLS(y1, X1).fit()

# Regresi OLS untuk subset kedua (2020-2022)
model2 = sm.OLS(y2, X2).fit()

# Gabungkan data df1 dan df2 untuk regresi gabungan (pooling)
df_combined = pd.concat([df1, df2])

# Variabel dependen gabungan
y_combined = df_combined['Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)']
X_combined = df_combined[[
        'Persentase Penduduk Miskin', 
        'Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010', 
        'Gini Ratio', 
        'Rata-Rata Lama Sekolah', 
        'TPT']]
X_combined = sm.add_constant(X_combined)  # Menambahkan konstanta (intercept)

# Regresi OLS untuk data gabungan
model_combined = sm.OLS(y_combined, X_combined).fit()

# Menghitung RSS (Residual Sum of Squares) untuk masing-masing model
RSS_pooled = model_combined.ssr  # RSS untuk model gabungan
RSS1 = model1.ssr  # RSS untuk model 1 (2018-2019)
RSS2 = model2.ssr  # RSS untuk model 2 (2020-2022)

# Jumlah parameter (termasuk konstanta) untuk setiap model
k = len(X_combined.columns)  
n1 = len(X1) 
n2 = len(X2)  

# Menghitung F-statistic Chow Test
F_stat = ((RSS_pooled - (RSS1 + RSS2)) / k) / ((RSS1 + RSS2) / (n1 + n2 - 2 * k))

# Menampilkan F-statistic
print("F-statistic for Chow Test:", F_stat)

# Menghitung nilai kritis F untuk tingkat signifikansi 0.05
alpha = 0.05
dfn = k  # Derajat kebebasan pembilang
dfd = n1 + n2 - 2 * k  # Derajat kebebasan penyebut
F_critical = f.ppf(1 - alpha, dfn, dfd)

print("Critical F value:", F_critical)

# Membandingkan F-statistic dengan nilai kritis
if F_stat > F_critical:
    print("Tolak H0: Ada perbedaan signifikan antara koefisien kedua kelompok.")
else:
    print("Gagal tolak H0: Tidak ada perbedaan signifikan antara koefisien kedua kelompok.")

F-statistic for Chow Test: 0.5760167563286263
Critical F value: 2.147597376520648
Gagal tolak H0: Tidak ada perbedaan signifikan antara koefisien kedua kelompok.


# BP-LM Test

In [7]:
# Menyiapkan data untuk model pooled OLS
df_panel = pd.concat([df1, df2])  # Gabungkan df1 dan df2 jika perlu
df_panel = df_combined.set_index(['Kabupaten/Kota', 'Tahun'])
y_panel = df_panel['Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)']
X_panel = df_panel[['Persentase Penduduk Miskin', 
                    'Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010', 
                    'Gini Ratio', 'Rata-Rata Lama Sekolah', 'TPT']]

# Model Pooled OLS
pooled_model = PooledOLS(y_panel, X_panel).fit()

# Model Random Effects
random_model = RandomEffects(y_panel, X_panel).fit()

# Membandingkan model
test_results = compare({'Pooled OLS': pooled_model, 'Random Effects': random_model})

# Menampilkan hasil perbandingan
print(test_results)

                                                                                        Model Comparison                                                                                        
                                                                                                                                   Pooled OLS                                     Random Effects
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable                                                                                Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)   Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)
Estimator                                                                                                                           PooledOLS                                      RandomEffects
No. Observations                   

In [8]:
# Model Pooled OLS
pooled_model = PooledOLS(y_panel, X_panel).fit()

# Model Random Effects
random_model = RandomEffects(y_panel, X_panel).fit()

# Uji Hausman
# Menghitung selisih koefisien
diff = pooled_model.params - random_model.params

# Menghitung kovarians dari perbedaan koefisien
cov_matrix = np.linalg.inv(pooled_model.cov) + np.linalg.inv(random_model.cov)

# Menghitung statistik uji Hausman
hausman_stat = diff.T @ np.linalg.inv(cov_matrix) @ diff

# Derajat kebebasan (jumlah parameter yang diestimasi)
dfn = len(pooled_model.params)

# Menghitung p-value dari statistik uji Hausman
p_value = 1 - stats.chi2.cdf(hausman_stat, dfn)

print(f'Hausman Test Statistic: {hausman_stat}')
print(f'P-value: {p_value}')

# Menentukan hasil uji
if p_value < 0.05:
    print("Tolak H0: Model Random Effects lebih tepat daripada Pooled OLS.")
else:
    print("Gagal tolak H0: Model Pooled OLS lebih tepat daripada Random Effects.")

NameError: name 'stats' is not defined

In [9]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.stats.diagnostic import het_breuschpagan

df_panel = pd.concat([df1, df2])  # Pastikan data sudah digabungkan dengan benar

# Set indeks untuk panel data: Kabupaten/Kota dan Tahun
df_panel = df_panel.set_index(['Kabupaten/Kota', 'Tahun'])

# Definisikan variabel dependen dan independen
y_panel = df_panel['Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)']
X_panel = df_panel[['Persentase Penduduk Miskin', 
                    'Laju Pertumbuhan Produk Domestik Regional Bruto per Kapita Atas Dasar Harga Konstan 2010', 
                    'Gini Ratio', 'Rata-Rata Lama Sekolah', 'TPT']]

# 1. Model Fixed Effects (FE)
fixed_effects = PanelOLS(y_panel, X_panel, entity_effects=True).fit()

# 2. Model Random Effects (RE)
random_effects = RandomEffects(y_panel, X_panel).fit()

# 3. Uji Hausman untuk membandingkan model Fixed Effects dan Random Effects
hausman_test = compare({'Fixed Effects': fixed_effects, 'Random Effects': random_effects})

# Menampilkan hasil uji Hausman
print(hausman_test)

                                                                                           Model Comparison                                                                                           
                                                                                                                                    Fixed Effects                                       Random Effects
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable                                                                                    Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)     Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)
Estimator                                                                                                                                PanelOLS                                        RandomEffects
No. O

# Haussman Test

In [10]:
# Fixed Effects
random_effects = RandomEffects(y_panel, X_panel).fit()
print(random_effects)

                                       RandomEffects Estimation Summary                                       
Dep. Variable:     Pengeluaran per Kapita Disesuaikan (Ribu Rupiah)   R-squared:                        0.8891
Estimator:                                            RandomEffects   R-squared (Between):              0.9814
No. Observations:                                               198   R-squared (Within):               0.4529
Date:                                              Fri, Nov 15 2024   R-squared (Overall):              0.9811
Time:                                                      17:18:22   Log-likelihood                   -1319.2
Cov. Estimator:                                          Unadjusted                                           
                                                                      F-statistic:                      309.51
Entities:                                                        33   P-value                           0.0000
A